In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vapeplot 
from scipy import interp
import scipy.stats
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
from datetime import datetime
def timestamp(): return datetime.today().strftime('%Y%m%d')

from sklearn.preprocessing import QuantileTransformer
def quantile_scale(df,feats):
    qua = df
    scaler = QuantileTransformer(
        n_quantiles=10,
        random_state=42,
        ignore_implicit_zeros=True, #sparse matrix
    )
    # fit the scaler
    scaler.fit(qua[feats])
    # transform values
    qua[feats] = scaler.transform(qua[feats])
    return qua
def tiered(classes):
    trans = []
    for x in classes:
        if x==1: c=0
        if x==2: c=1
        if x>=3 and x<=4: c=2
        if x>=5 and x<=7: c=3
        if x>=8: c=4
        trans.append(c)
    return trans

In [3]:
EPI=9
merge_col = ['season','baker','index','episode','place']
tech = pd.read_csv("../../DATA/s11/deepbake_s11_technical_features.20201120.tsv",sep="\t")
star = pd.read_csv("../../DATA/s11/deepbake_judge_features_s11_e9.20201120.tsv",sep="\t")
gbbo = pd.merge(tech, star,  how='left', left_on=merge_col, right_on =merge_col)
gbbo = gbbo[['season','baker','episode','place','tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']]
gbbo.to_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t",index=False)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
0,11,Dave,1,0,0.0,0,0.00,0.0,0.00,0.0,0.00,0.0
1,11,Dave,2,0,0.0,0,0.50,1.0,0.50,1.0,0.00,0.0
2,11,Dave,3,0,0.0,0,0.33,0.0,0.33,0.0,0.33,1.0
3,11,Dave,4,0,0.0,0,0.25,0.0,0.25,0.0,0.25,0.0
4,11,Dave,5,0,0.0,0,0.20,0.0,0.40,1.0,0.20,0.0


In [5]:
gbbo = pd.read_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t")
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
max_epi = EPI
gbbo = gbbo.loc[gbbo['episode']==max_epi]
gbbo = quantile_scale(gbbo,feats)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
8,11,Dave,9,0,0.186744,0.909091,0.666667,0.0,0.888889,1.0,0.000000,0.0
18,11,Hermine,9,0,0.741184,0.818182,1.000000,0.0,0.888889,0.0,0.611111,1.0
28,11,Laura,9,0,0.923434,1.000000,0.666667,0.0,0.555556,0.0,0.888889,1.0
38,11,Linda,9,0,0.540404,0.000000,0.000000,0.0,0.000000,0.0,0.888889,0.0
48,11,Loriea,9,1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [6]:
tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
tech = tech.loc[tech['episode']==max_epi]
qua = quantile_scale(tech,feats)
qua['place']=tiered(qua['place'])
X, y = np.matrix(qua[feats]), np.array(qua['place'])

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.activations import relu

def create_model( nl1=1, nl2=1,  nl3=1, 
                 nn1=1000, nn2=500, nn3 = 200, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0,input_shape=None,output_shape=None):    
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture
    https://www.kaggle.com/arrogantlymodest/randomised-cv-search-over-keras-neural-network
    '''
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
    model = Sequential()
    first=True  
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))       
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model
##################################
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
tech['place']=tiered(tech['place'])
input_shape = len(feats)
output_shape = len(set(tech['place']))


l1 = 0.0001
l2 = 0.0001
lr = 0.0001
nl1 = 1
nl2 = 1
nl3 = 1
nn1 = 800
nn2 = 800
nn3 = 300

dropout = 0.1
decay = 1e-09
act='relu'
n_dims = len(feats)
n_classes = len(set(tech['place']))


BATCH,EPOCHS = 12, 25

In [8]:
GBBO = pd.read_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t")
S11 = pd.DataFrame()
GBBO = GBBO.loc[GBBO['episode']<=EPI]
for e in set(GBBO['episode']):
    gbbo = GBBO.loc[GBBO['episode']==e]
    gbbo = quantile_scale(gbbo,feats)
    test = np.matrix(gbbo[feats])

    tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
    tech = tech.loc[tech['episode']==e]
    qua = quantile_scale(tech,feats)
    qua['place']=tiered(qua['place'])
    
    X, y = np.matrix(qua[feats]), np.array(qua['place'])
    
    nn = create_model( nl1=nl1, nl2=nl2,  nl3=nl3, 
                     nn1=nn1, nn2=nn2, nn3 = nn3, 
                     lr=lr, decay=decay, l1=l1, l2=l2,
                     act = act, dropout=dropout,
                     input_shape=n_dims,
                     output_shape=n_classes)
    
    nn.fit(X,y,validation_split=0., batch_size=BATCH, epochs=EPOCHS,verbose=0)

    preds = nn.predict_classes(test)
    probs = nn.predict_proba(test)
    gbbo['preds']=preds
    # probability baker is a finalist
    top = probs[:,0]
    # probability baker is a finalist or a runner-up
    top3 = probs[:,0]+probs[:,1]
    # bottom tier (8th and below)
    bot = probs[:,-1]
    # 5th - 7th
    nextbot = probs[:,-2]
    third = probs[:,-3]

    gbbo['bottom']=np.round(bot*100,decimals=2)
    gbbo['finalist']=np.round(top*100,decimals=2) 
    gbbo['top3'] = np.round(top3*100,decimals=2)
    gbbo['fifthseventh'] = np.round(nextbot*100,decimals=2)
    gbbo['thirdforth'] = np.round(third*100,decimals=2)
    S11 = pd.concat([S11,gbbo])
    gbbo.to_csv("deepbake_s11.week{}_preditions.txt".format(e),sep="\t",index=False)
    print(e)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use `model.predict()` instead.
1
2
3
4
5
6
7
8
9


In [9]:
gbbo.sort_values(['top3'],ascending=False)

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad,preds,bottom,finalist,top3,fifthseventh,thirdforth
98,11,Peter,9,0,0.623457,0.727273,1.000000,1.0,0.727273,1.0,0.611111,0.0,1,6.410000,22.059999,85.010002,4.290000,4.280000
8,11,Dave,9,0,0.186744,0.909091,0.666667,0.0,0.888889,1.0,0.000000,0.0,1,6.740000,25.559999,81.470001,5.790000,6.000000
18,11,Hermine,9,0,0.741184,0.818182,1.000000,0.0,0.888889,0.0,0.611111,1.0,1,3.910000,21.549999,60.150002,5.310000,30.629999
28,11,Laura,9,0,0.923434,1.000000,0.666667,0.0,0.555556,0.0,0.888889,1.0,2,5.860000,13.120000,30.350000,8.320000,55.470001
58,11,Lottie,9,0,0.802182,0.000000,0.666667,0.0,0.363636,0.0,0.000000,0.0,3,18.990000,7.730000,18.379999,42.779999,19.840000
118,11,Sura,9,4,0.310279,0.000000,0.000000,0.0,0.555556,0.0,0.000000,0.0,3,26.750000,9.100000,17.870001,34.529999,20.850000
48,11,Loriea,9,1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.810001,9.490000,17.639999,24.719999,13.840000
68,11,Mak,9,2,0.047210,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.560001,9.110000,16.930000,25.760000,13.750000
88,11,Mark,9,0,0.365591,0.000000,0.666667,0.0,0.555556,0.0,0.611111,0.0,3,32.160000,3.280000,6.390000,45.200001,16.250000
78,11,Marc,9,0,1.000000,0.000000,0.666667,0.0,0.888889,0.0,0.888889,0.0,3,19.420000,0.890000,1.750000,64.620003,14.210000


In [10]:
gbbo.sort_values(['thirdforth'],ascending=False)

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad,preds,bottom,finalist,top3,fifthseventh,thirdforth
28,11,Laura,9,0,0.923434,1.000000,0.666667,0.0,0.555556,0.0,0.888889,1.0,2,5.860000,13.120000,30.350000,8.320000,55.470001
18,11,Hermine,9,0,0.741184,0.818182,1.000000,0.0,0.888889,0.0,0.611111,1.0,1,3.910000,21.549999,60.150002,5.310000,30.629999
118,11,Sura,9,4,0.310279,0.000000,0.000000,0.0,0.555556,0.0,0.000000,0.0,3,26.750000,9.100000,17.870001,34.529999,20.850000
58,11,Lottie,9,0,0.802182,0.000000,0.666667,0.0,0.363636,0.0,0.000000,0.0,3,18.990000,7.730000,18.379999,42.779999,19.840000
88,11,Mark,9,0,0.365591,0.000000,0.666667,0.0,0.555556,0.0,0.611111,0.0,3,32.160000,3.280000,6.390000,45.200001,16.250000
78,11,Marc,9,0,1.000000,0.000000,0.666667,0.0,0.888889,0.0,0.888889,0.0,3,19.420000,0.890000,1.750000,64.620003,14.210000
48,11,Loriea,9,1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.810001,9.490000,17.639999,24.719999,13.840000
68,11,Mak,9,2,0.047210,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.560001,9.110000,16.930000,25.760000,13.750000
8,11,Dave,9,0,0.186744,0.909091,0.666667,0.0,0.888889,1.0,0.000000,0.0,1,6.740000,25.559999,81.470001,5.790000,6.000000
108,11,Rowan,9,3,0.449495,0.000000,0.000000,0.0,0.000000,0.0,0.611111,0.0,4,63.340000,0.970000,1.630000,30.379999,4.650000


In [11]:
gbbo.sort_values(['finalist'],ascending=False)

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad,preds,bottom,finalist,top3,fifthseventh,thirdforth
8,11,Dave,9,0,0.186744,0.909091,0.666667,0.0,0.888889,1.0,0.000000,0.0,1,6.740000,25.559999,81.470001,5.790000,6.000000
98,11,Peter,9,0,0.623457,0.727273,1.000000,1.0,0.727273,1.0,0.611111,0.0,1,6.410000,22.059999,85.010002,4.290000,4.280000
18,11,Hermine,9,0,0.741184,0.818182,1.000000,0.0,0.888889,0.0,0.611111,1.0,1,3.910000,21.549999,60.150002,5.310000,30.629999
28,11,Laura,9,0,0.923434,1.000000,0.666667,0.0,0.555556,0.0,0.888889,1.0,2,5.860000,13.120000,30.350000,8.320000,55.470001
48,11,Loriea,9,1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.810001,9.490000,17.639999,24.719999,13.840000
68,11,Mak,9,2,0.047210,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,4,43.560001,9.110000,16.930000,25.760000,13.750000
118,11,Sura,9,4,0.310279,0.000000,0.000000,0.0,0.555556,0.0,0.000000,0.0,3,26.750000,9.100000,17.870001,34.529999,20.850000
58,11,Lottie,9,0,0.802182,0.000000,0.666667,0.0,0.363636,0.0,0.000000,0.0,3,18.990000,7.730000,18.379999,42.779999,19.840000
88,11,Mark,9,0,0.365591,0.000000,0.666667,0.0,0.555556,0.0,0.611111,0.0,3,32.160000,3.280000,6.390000,45.200001,16.250000
108,11,Rowan,9,3,0.449495,0.000000,0.000000,0.0,0.000000,0.0,0.611111,0.0,4,63.340000,0.970000,1.630000,30.379999,4.650000
